## 1. Introduction
PP-YOLOE+ is a upgraded version of PP-YOLOE. Starting with a large-scale obj365 object detection pretraining model, PP-YOLOE+ not only greatly improves the convergence speed, but also improves the speed of the model on the COCO dataset. At the same time, PP-YOLOE+ greatly improves the end-to-end prediction speed including data preprocessing. For more details, please refer to [official documentation](https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.5/configs/ppyoloe/README.md)

## 2. Model Effects
PP-YOLOE+_l achieves 53.3 mAP on COCO test-dev2017 dataset with 78.1 FPS on Tesla V100. While using TensorRT FP16, PP-YOLOE+_l can be further accelerated to 149.2 FPS. PP-YOLOE+_s/m/x also have excellent accuracy and speed performance as shown below.
<div align="center">
  <img src="https://raw.githubusercontent.com/PaddlePaddle/PaddleDetection/release/2.5/docs/images/ppyoloe_plus_map_fps.png" width=500 />
</div>

## 3. How to use the model
Clone PaddleDetection firstly and put the COCO-style dataset in `dataset/coco`

In [ ]:
%mkdir -p ~/work
%cd ~/work/
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

%cd PaddleDetection
%mkdir -p demo_input demo_output
!pip install -r requirements.txt

### 3.1 Training
Training PP-YOLOE+ on 8 GPUs with following command

In [ ]:
# training with single GPU
!python tools/train.py -c configs/ppyoloe/ppyoloe_plus_crn_l_80e_coco.yml --eval --amp

# training with mutiple GPUs
!python -m paddle.distributed.launch --gpus 0,1,2,3,4,5,6,7 tools/train.py -c configs/ppyoloe/ppyoloe_plus_crn_l_80e_coco.yml --eval --amp

**Notes:**
- If you need to evaluate while training, please add `--eval`.
- PP-YOLOE supports mixed precision training, please add `--amp`.

### 3.2 Inference
#### 3.2.1 Inference with Paddle-TRT
Next, we will introduce how to use Paddle Inference to deploy PP-YOLOE+ models in TensorRT FP16 mode.

First, refer to [Paddle Inference Docs](https://www.paddlepaddle.org.cn/inference/master/user_guides/download_lib.html#python), download and install packages corresponding to CUDA, CUDNN and TensorRT version.

Then, Exporting PP-YOLOE+ for Paddle Inference **with TensorRT**, use following command.

In [ ]:
!python tools/export_model.py -c configs/ppyoloe/ppyoloe_plus_crn_l_80e_coco.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_plus_crn_l_80e_coco.pdparams trt=True

Finally, inference in TensorRT FP16 mode.

In [ ]:
# inference single image
!wget -P demo_input -N https://paddledet.bj.bcebos.com/modelcenter/images/General/000000014439.jpg
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyoloe_plus_crn_l_80e_coco --image_file=demo_input/000000014439.jpg --device=gpu --run_mode=trt_fp16 --output_dir=demo_output

*Notes:**
- TensorRT will perform optimization for the current hardware platform according to the definition of the network, generate an inference engine and serialize it into a file. This inference engine is only applicable to the current hardware hardware platform. If your hardware and software platform has not changed, you can set `use_static=True` in [enable_tensorrt_engine](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/deploy/python/infer.py#L745). In this way, the serialized file generated will be saved in the `output_inference` folder, and the saved serialized file will be loaded the next time when TensorRT is executed.
- PaddleDetection release/2.4 and later versions will support NMS calling TensorRT, which requires PaddlePaddle release/2.3 and later versions.

#### 3.2.2 Inference with PaddleInference
For some AI acceleration hardware that does not support TensorRT, we can directly use PaddleInference to deploy. First run the following command to export the model

In [ ]:
!python tools/export_model.py -c configs/ppyoloe/ppyoloe_plus_crn_l_80e_coco.yml -o weights=https://paddledet.bj.bcebos.com/models/ppyoloe_plus_crn_l_80e_coco.pdparams

Inference with PaddleInference directly.

In [ ]:
# inference single image
!CUDA_VISIBLE_DEVICES=0 python deploy/python/infer.py --model_dir=output_inference/ppyoloe_plus_crn_l_80e_coco --image_file=demo_input/000000014439.jpg --device=gpu --run_mode=paddle --output_dir=demo_output

## 4. Model principle
The overall structure of PP-YOLOE+ is roughly the same as that of PP-YOLOE. PP-YOLOE+ improves the preformance with the following methods:
- Pre training model using large-scale data set obj365
- In the backbone, add the alpha parameter to the block branch
- Optimize the end-to-end inference speed and improve the training convergence speed


## 5. Attention
**All commands run on AI Studio's `jupyter` by default. If running on a terminal, remove the % or ! at the beginning of the command.**

## 6. Related papers and citations
```
@article{xu2022pp,
  title={PP-YOLOE: An evolved version of YOLO},
  author={Xu, Shangliang and Wang, Xinxin and Lv, Wenyu and Chang, Qinyao and Cui, Cheng and Deng, Kaipeng and Wang, Guanzhong and Dang, Qingqing and Wei, Shengyu and Du, Yuning and others},
  journal={arXiv preprint arXiv:2203.16250},
  year={2022}
}
```